In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from splinter import Browser

In [2]:
def scrape_html_soup(seed_url, soup=True, click_css=''):
    executable_path = {'executable_path': 'C:/Users/markg/chromedriver/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False)
    # Visit the seed url
    browser.visit(seed_url)
    time.sleep(1)
    if click_css != '':
        browser.find_by_css(click_css).first.click()
        time.sleep(1)
    html = browser.html
    browser.quit()
    if soup: ## Scrape page into soup
        return BeautifulSoup(html, 'html.parser')
    else:
        return html

In [3]:
def scrape_mars_news():
    seed_url = 'https://mars.nasa.gov/news/'
    soup = scrape_html_soup(seed_url)
    article_title = soup.find('div', class_='content_title').text
    article_paragraph = soup.find('div', class_='article_teaser_body').text
    return {'articleTitle': article_title, 'articleParagraph': article_paragraph}
scraped_data = scrape_mars_news()
print(scraped_data['articleTitle'])
print(scraped_data['articleParagraph'])

NASA's Next Mars Lander Spreads its Solar Wings
NASA's next mission to Mars passed a key test Tuesday, extending the solar arrays that will power the InSight spacecraft once it lands on the Red Planet this November.


In [4]:
def scrape_featured_mars_image():
    seed_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/'
    soup = scrape_html_soup(seed_url, click_css='div.carousel_container div.carousel_items a.button')
    landing_page_url = seed_url[:seed_url.find('/',8)]
    featured_image_url = soup.find('img', class_='fancybox-image')['src']
    featured_image_url = f'{landing_page_url}{featured_image_url}'
    featured_image_url = featured_image_url.replace('medium', 'large').replace('ip', 'hires')
    return {'featuredImageUrl': featured_image_url}
scraped_data = scrape_featured_mars_image()
scraped_data['featuredImageUrl']

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15283_hires.jpg'

In [5]:
def scrape_mars_weather_tweet():
    seed_url = 'https://twitter.com/marswxreport?lang=en'
    soup = scrape_html_soup(seed_url)
    mars_weather_tweet = soup.find('div', attrs={'class': 'tweet', 'data-name': 'Mars Weather'})\
        .find('p', attrs={'class': 'tweet-text'}).text
    return {'marsWeatherTweet': mars_weather_tweet}
scraped_data = scrape_mars_weather_tweet()
scraped_data['marsWeatherTweet']

'Sol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28'

In [6]:
def scrape_mars_facts_html():
    seed_url = 'http://space-facts.com/mars/'
    html = scrape_html_soup(seed_url, soup=False)
    mars_facts_df = pd.read_html(html, attrs = {'id': 'tablepress-mars'})[0]
    mars_facts_df.columns = ['Measurement', 'Value']
    mars_facts_df['Measurement'] = [measurement[:-1] for measurement in mars_facts_df['Measurement']]
    mars_facts_html = mars_facts_df.to_html()
    return {'marsFactsHtml': mars_facts_html}
scraped_data = scrape_mars_facts_df()
scraped_data['marsFactsHtml']

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measurement</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record</td>\n      <td>2nd millennium BC</td>